In [1]:
import os, json, asyncio
from pathlib import Path
from playwright.async_api import async_playwright
import pandas as pd
from IPython.display import clear_output

In [2]:
df = pd.read_parquet("datos/2025.parquet")

In [3]:
df = df[df.nuevo]

In [4]:
nro_documentos = df.nro_documento.tolist()

In [5]:
CHECKPOINT = Path("sigep_checkpoint.json")

In [6]:
async def run(nro_documentos):
    async with async_playwright() as p:
        if CHECKPOINT.exists():
            with open(CHECKPOINT, "r") as f:
                done, data = json.load(f)
        else:
            done, data = [], []

        remaining = [c for c in nro_documentos if c not in done]
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        await page.goto(
            "https://sigep.gob.bo/rsportaltransparencia/#/portalHome/personal"
        )
        await page.wait_for_load_state("networkidle")
        await page.click("a.pull-right[type=button]")
        for nro_documento in remaining:
            clear_output(wait=True)
            print(f"{len(done)} / {len(nro_documentos)} documentos ({len(data)} registros): {nro_documento}")
            try:
                await page.fill("input[name=ci]", str(nro_documento))
                async with page.expect_response(
                    "**/persona/lista?tipoBusqueda=AP&**", timeout=30000
                ) as response_info:
                    await page.click("input[value=Buscar]")
                response = await response_info.value
                if response:
                    response_data = await response.json()
                if response_data["data"]["lista"]:
                    data.append(
                        {
                            **{"nro_documento": nro_documento},
                            **response_data["data"]["lista"][0],
                        }
                    )
                done = done + [nro_documento]
                await asyncio.sleep(2)
            except Exception as e:
                print(e)
                break
            with open(CHECKPOINT, "w") as f:
                json.dump([done, data], f, ensure_ascii=False, indent=2)

        await browser.close()

In [7]:
await run(nro_documentos)

374 / 375 documentos (20 registros): 4200980


In [8]:
with open(CHECKPOINT, 'r') as f:
    sigep = pd.DataFrame(json.load(f)[1])

In [10]:
old = pd.read_parquet("datos/sigep.parquet")

In [15]:
sigep = pd.concat([old, sigep])

In [16]:
sigep

,nro_documento,id,memorandum,entidad,idEntidad,siglaEntidad,descEntidad,descUrh,descUnidad,nombres,primerApellido,segundoApellido,razonSocial,descCargo,descPuesto,descModalidad,monto,fechaIni,fechaFin,apiEstado
0,7971986,390688,390,650,575,ALP,Asamblea Legislativa Plurinacional,CAMARA DE SENADORES,SENADORES TITULARES BRIGADA COCHABAMBA,ANDRONICO,RODRIGUEZ,LEDEZMA,RODRIGUEZ LEDEZMA ANDRONICO,SENADORES TITULARES,SENADOR TITULAR,PERSONAL DE PLANTA,23311.0,01/05/2024,None,Vigente
1,10667840,391054,334,169,403,AIT,Autoridad de Impugnación Tributaria,RRHH AIT,SECRETARIA DE CAMARA,CARLA FAVIOLA,FLORES,AGUIRRE,FLORES AGUIRRE CARLA FAVIOLA,ANALISTA I - SECRETARIO CÁMARA II,ANALISTA DE INFORMACION LEGAL Y DILIGENCIAS,PERSONAL DE PLANTA,13049.0,01/05/2024,None,Vigente
2,6164154,365509,286,680,579,CGE,Contraloria General del Estado,GNRH CONTRALORÍA GENERAL DEL ESTADO,SUBCONTRALORÍA DE SERVICIOS LEGALES,SANDRA,HINOJOSA,NUÑEZ,HINOJOSA NUÑEZ SANDRA,AUXILIAR I,AUXILIAR,PERSONAL DE PLANTA,5263.0,01/05/2024,None,Vigente
3,6180723,428952,4069,292,487,VOBO,Vías Bolivia,URH VIAS BOLIVIA OFICINA NACIONAL,OFICINA REGIONAL LA PAZ,HERLAN PAOOLO,None,PACHECO,PACHECO HERLAN PAOOLO,TÉCNICO IV,CHOFER LA PAZ,PERSONAL DE PLANTA,4469.0,01/05/2024,None,Vigente
4,7061347,576244,3139,340,358,SEGIP,Servicio General de Identificación Personal,URH - ENTIDAD 340 SEGIP,DIRECCION NACIONAL DE COMUNICACION,PAOLA,HUARACHI,MAMANI,HUARACHI MAMANI PAOLA,TECNICO I,TECNICO COMUNICACION,PERSONAL DE PLANTA,5729.0,01/05/2024,None,Vigente
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15,2206723,526942,352,1205,50,ELLA,Gobierno Autónomo Municipal de El Alto de La Paz,ADMINISTRACION CENTRAL - EJECUTIVO,UNIDAD DE PAVIMENTOS,GERMAN,VALDIVIA,CHOQUE,VALDIVIA CHOQUE GERMAN,JEFE C / RESPONSABLE DE AREA I,JEFE(A) DE UNIDAD,PERSONAL DE PLANTA,8790.0,03/04/2023,None,Vigente
16,7742946,360868,2265,342,591,AEVIVIENDA,Agencia Estatal de Vivienda,URH 342 AEVIVIENDA,DIRECCION DEPARTAMENTAL DE COCHAMBAMBA,JHIMY DAVID,TORRICO,HINOJOSA,TORRICO HINOJOSA JHIMY DAVID,TECNICO II,TECNICO II EN SISTEMAS,PERSONAL DE PLANTA,9216.0,17/06/2024,None,Vigente
17,5537960,364252,2931,650,575,ALP,Asamblea Legislativa Plurinacional,CAMARA DE DIPUTADOS,ASAMBLEA LEGISLATIVA PLURINACIONAL DE BOLIVIA,JUAN JOSE,TORREZ,FLORES,TORREZ FLORES JUAN JOSE,DIPUTADOS TITULARES,DIPUTADOS TITULARES,PERSONAL DE PLANTA,23310.0,01/05/2024,None,Vigente
18,6488972,373651,361,1301,133,CBB,Gobierno Autónomo Municipal de Cochabamba,URH 1 CONCEJO MUNICIPAL,PLENO,DANIELA,CABRERA,GUILLEN,CABRERA GUILLEN DANIELA,CONCEJAL,CONCEJAL,PERSONAL DE PLANTA,18380.0,01/05/2024,None,Vigente


In [17]:
sigep.to_parquet("datos/sigep.parquet")